In [69]:
from importlib import reload

import treeflow.tree_processing
reload(treeflow.tree_processing)
import treeflow.sequences
newick_file = '../data/analysis-tree.nwk'
fasta_file = '../data/sim-seq.fasta'

tree, taxon_names = treeflow.tree_processing.parse_newick(newick_file)
topology = treeflow.tree_processing.update_topology_dict(tree['topology'])
taxon_count = len(taxon_names)
taxon_count, topology.keys()

(100,
 dict_keys(['postorder_node_indices', 'child_indices', 'preorder_indices', 'preorder_node_indices', 'sibling_indices', 'parent_indices']))

# Variational approximation

In [124]:
height_dist.sample([])[100:]

<tf.Tensor: shape=(99,), dtype=float32, numpy=
array([3.1539671e-02, 3.1707361e-02, 3.3669174e-02, 3.4416940e-02,
       3.9227877e-02, 1.5278661e-01, 2.9281050e-02, 2.9817760e-02,
       3.7698522e-02, 4.8394915e-02, 1.5825930e-01, 2.6414979e-02,
       7.7069672e-03, 6.1014064e-02, 2.0218240e-01, 1.4398035e-04,
       1.4903551e-04, 2.7377141e-04, 6.4432633e-04, 5.5112112e-03,
       2.5158632e-03, 3.3314901e-03, 1.5429773e-04, 2.3408523e-03,
       3.3783810e-03, 4.7703835e-04, 3.4068779e-03, 4.7521075e-04,
       7.9969846e-04, 4.3903253e-04, 1.1855022e-03, 4.8249131e-03,
       2.2244620e-05, 4.4834309e-05, 9.2090648e-03, 1.0347679e-02,
       1.0375109e-02, 1.0400512e-02, 1.6186960e-02, 2.5360838e-01,
       3.5788448e-04, 2.6304787e-03, 2.6559769e-03, 5.1066480e-03,
       5.2306899e-03, 7.5522349e-03, 1.6772324e-03, 4.7495305e-03,
       5.0779567e-05, 7.6526627e-03, 8.6520808e-03, 2.0652513e-03,
       1.5676182e-02, 4.5067526e-04, 9.9393108e-04, 2.0179862e-02,
       6.737632

In [108]:
import treeflow.tree_transform
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions


import treeflow.tf_util
reload(treeflow.tf_util)

reload(treeflow.tree_transform)

anchor_heights = treeflow.tree_processing.get_node_anchor_heights(tree['heights'], topology['postorder_node_indices'], topology['child_indices'])
anchor_heights = tf.convert_to_tensor(anchor_heights, dtype=tf.float32)
tree_chain = treeflow.tree_transform.TreeChain(
    topology['parent_indices'][taxon_count:] - taxon_count,
    topology['preorder_node_indices'][1:] - taxon_count,
    anchor_heights=anchor_heights)
init_heights = tf.convert_to_tensor(tree['heights'][taxon_count:], dtype=tf.float32)
init_heights_trans = tree_chain.inverse(init_heights)
leaf_heights = tf.convert_to_tensor(tree['heights'][:taxon_count], dtype=tf.float32)

height_dist = tfd.Blockwise([
    tfd.Independent(tfd.Deterministic(leaf_heights), reinterpreted_batch_ndims=1),
    tfd.TransformedDistribution(
        distribution=tfd.Independent(tfd.Normal(
            loc=tf.Variable(init_heights_trans, name='q_tree_loc'),
            scale=tfp.util.DeferredTensor(tf.Variable(tf.ones_like(init_heights_trans), name='q_tree_scale'), tf.nn.softplus)
        ), reinterpreted_batch_ndims=1),
        bijector=tree_chain
    )
])

q = tfd.JointDistributionNamed(dict(
    tree=treeflow.tree_transform.FixedTopologyDistribution(
        height_distribution=height_dist,
        topology=tree['topology']
    ),
    kappa=tfd.LogNormal(
        loc=tf.Variable(0.0, name='q_kappa_loc'),
        scale=tfp.util.DeferredTensor(tf.Variable(1.0, name='q_kappa_scale'), tf.nn.softplus)
    ),
    pop_size=tfd.LogNormal(
        loc=tf.Variable(0.0, name='q_pop_size_loc'),
        scale=tfp.util.DeferredTensor(tf.Variable(1.0, name='q_pop_size_scale'), tf.nn.softplus)
    ),
    frequencies=tfd.Dirichlet(concentration=tfp.util.DeferredTensor(tf.Variable([4.0, 4.0, 4.0, 4.0], name='q_frequencies_concentration'), tf.nn.softplus))
))
#[(var.name, var.shape) for var in q.trainable_variables]
q.sample()

{'tree': {'topology': {'parent_indices': <tf.Tensor: shape=(198,), dtype=int64, numpy=
   array([100, 100, 101, 102, 103, 104, 105, 106, 106, 107, 108, 109, 111,
          111, 112, 112, 115, 115, 116, 117, 118, 118, 120, 120, 121, 122,
          122, 123, 125, 125, 127, 127, 128, 128, 129, 129, 130, 132, 132,
          133, 136, 140, 140, 141, 141, 142, 144, 145, 146, 146, 147, 148,
          148, 151, 151, 153, 153, 154, 156, 156, 157, 158, 158, 159, 161,
          161, 162, 164, 165, 166, 167, 167, 168, 169, 170, 173, 174, 174,
          175, 176, 177, 177, 179, 181, 181, 183, 183, 184, 185, 185, 186,
          186, 187, 188, 189, 190, 192, 192, 193, 194, 101, 102, 103, 104,
          105, 110, 107, 108, 109, 110, 114, 113, 113, 114, 139, 116, 117,
          119, 119, 138, 121, 124, 123, 124, 126, 126, 137, 135, 131, 130,
          131, 134, 133, 134, 135, 136, 137, 138, 139, 198, 143, 142, 143,
          144, 145, 150, 147, 149, 149, 150, 152, 152, 155, 154, 155, 172,
          157

In [105]:
q.sample()['tree']['heights'][100:]

<tf.Tensor: shape=(99,), dtype=float32, numpy=
array([2.4701494e-01, 2.5051275e-01, 2.5805616e-01, 2.6389703e-01,
       1.2491179e+00, 1.2598920e+00, 5.1959977e-02, 5.3044010e-02,
       1.3607390e-01, 1.9051033e-01, 1.2734506e+00, 8.2185483e-01,
       6.2938839e-02, 8.7663782e-01, 1.3883339e+00, 6.2352832e-02,
       6.2751278e-02, 8.4064581e-02, 7.4938230e-02, 2.1831986e-01,
       1.0403266e-01, 2.4199279e-01, 1.4445517e-02, 1.9211520e-01,
       2.4791346e-01, 9.1138603e-03, 2.5324798e-01, 6.9000679e-03,
       9.7703710e-03, 1.4617580e-03, 9.0584056e-03, 4.1126110e-02,
       2.2871705e-04, 1.5981584e-03, 1.3067338e-01, 2.6684505e-01,
       2.6752871e-01, 2.6927808e-01, 3.6579776e-01, 1.4862388e+00,
       5.9798913e-05, 1.9644253e-04, 2.2881065e-04, 6.7917989e-03,
       6.8735857e-03, 6.9430331e-03, 5.9544323e-03, 7.9437215e-03,
       2.3122560e-04, 1.2245171e-02, 1.8292863e-02, 1.1881910e-03,
       1.8488785e-02, 8.0205789e-03, 1.0253385e-02, 8.3345525e-02,
       7.135687

In [109]:
q_sample['tree']['heights'][:, 100:]

<tf.Tensor: shape=(2, 99), dtype=float32, numpy=
array([[0.00000000e+00, 5.40370196e-02, 5.56325652e-02, 5.59946708e-02,
        9.71427932e-02, 1.09501719e-01, 5.55291921e-02, 6.17635213e-02,
        7.41885975e-02, 7.77562931e-02, 1.36586905e-01, 1.96162865e-01,
        8.39406550e-02, 2.06706226e-01, 5.30979753e-01, 2.04794784e-03,
        2.07947195e-03, 4.84392699e-03, 2.89720669e-03, 1.43276781e-01,
        9.43046808e-02, 3.06958348e-01, 5.95356226e-02, 2.89564699e-01,
        3.11019272e-01, 1.79498002e-01, 3.59149456e-01, 1.58389658e-01,
        1.25046158e-02, 2.08654515e-02, 2.37081684e-02, 1.17660820e-01,
        1.63848512e-02, 2.02537104e-02, 2.59631842e-01, 3.62676233e-01,
        3.63451779e-01, 3.72169137e-01, 4.11159933e-01, 6.44824028e-01,
        1.28994114e-03, 4.30126786e-02, 4.34185527e-02, 5.09298034e-02,
        5.10673672e-02, 5.27621210e-02, 7.95562565e-03, 3.34477536e-02,
        4.32543573e-04, 5.39242513e-02, 5.70423715e-02, 3.77675742e-02,
        8.60176

In [44]:
tree_chain.inverse_log_det_jacobian(q_sample['tree']['heights'][:, 100:], 1)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([nan, nan], dtype=float32)>

In [42]:
q_sample = q.sample(sample_shape=[2])
tree_chain.inverse(q_sample['tree']['heights'][:, 100:])

<tf.Tensor: shape=(2, 99), dtype=float32, numpy=
array([[            nan,  3.52226877e+00,  5.03440905e+00,
         3.07899654e-01,  2.06170034e+00,  1.39686203e+00,
         2.18683791e+00,  1.60359633e+00,  3.03455567e+00,
         2.78788567e-01, -1.06045961e+00,  2.92334676e+00,
        -3.80404830e-01, -4.50385511e-01,  1.53985465e+00,
         4.16402721e+00, -2.94393718e-01, -3.35679698e+00,
        -3.88753009e+00, -6.25912189e-01, -8.13240051e-01,
         4.32527018e+00, -1.35179877e+00,  2.60237169e+00,
         1.86588120e+00, -9.10162926e-04,  3.31722045e+00,
        -2.54591882e-01, -2.13014913e+00,  1.99237549e+00,
        -1.37781525e+00, -1.87986791e-01,  1.44217610e+00,
        -2.47069430e+00,  9.24027801e-01,  6.14763689e+00,
         3.73027444e+00,  2.25596905e+00,  5.65049231e-01,
        -5.47991633e-01, -3.65798020e+00,  4.66274023e+00,
         1.75402379e+00,  5.91371584e+00,  3.40521741e+00,
         2.51140332e+00, -1.16574812e+00,  4.90408003e-01,
       

In [32]:
q.log_prob_parts(q.sample(sample_shape=[2]))

{'tree': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([nan, nan], dtype=float32)>,
 'pop_size': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([-0.5874181, -0.7373816], dtype=float32)>,
 'kappa': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([-0.78352123, -4.596287  ], dtype=float32)>,
 'frequencies': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([2.3719292, 3.7056866], dtype=float32)>}

In [23]:
q.log_prob_parts(q.sample())

{'tree': <tf.Tensor: shape=(), dtype=float32, numpy=nan>,
 'pop_size': <tf.Tensor: shape=(), dtype=float32, numpy=nan>,
 'kappa': <tf.Tensor: shape=(), dtype=float32, numpy=-1.2846406>,
 'frequencies': <tf.Tensor: shape=(), dtype=float32, numpy=3.9390087>}

# Model
## Prior

In [3]:
import treeflow.coalescent
reload(treeflow.coalescent)

sampling_times = tf.convert_to_tensor(tree['heights'][:taxon_count], dtype=tf.float32)

prior = tfd.JointDistributionNamed(dict(
    frequencies=tfd.Dirichlet(concentration=[4,4,4,4]),
    kappa=tfd.LogNormal(loc=0, scale=1),
    pop_size=tfd.LogNormal(loc=0, scale=1),
    #site_alpha=tfd.LogNormal(loc=0, scale=1),
    #clock_rate=tfd.LogNormal(loc=0, scale=1),
    tree=lambda pop_size: treeflow.coalescent.ConstantCoalescent(pop_size=pop_size, sampling_times=sampling_times)
))

prior.log_prob_parts(q.sample(sample_shape=[2]))

{'pop_size': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([-2.0272362, -9.440367 ], dtype=float32)>,
 'tree': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([-217.77231, -326.9209 ], dtype=float32)>,
 'kappa': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([-0.6592641, -3.6463604], dtype=float32)>,
 'frequencies': <tf.Tensor: shape=(2,), dtype=float32, numpy=array([2.6924534, 3.4355507], dtype=float32)>}

In [9]:
q.log_prob(q.sample())

<tf.Tensor: shape=(), dtype=float32, numpy=nan>

In [8]:
def kl():
    q_samples = q.sample()
    return q.log_prob(q_sample) - prior.log_prob(q_samples)

kl()

<tf.Tensor: shape=(), dtype=float32, numpy=nan>

In [4]:
tfp.vi.fit_surrogate_posterior(prior.log_prob, q, tf.optimizers.Adam(), 10)

/mnt/c/chris/git/treeflow/treeflow/coalescent.py:75: UserWarning: Dummy sampling
  warnings.warn('Dummy sampling')


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)>

In [5]:
import treeflow.vi
from importlib import reload
reload(treeflow.vi)

treeflow.vi.fit_surrogate_posterior(prior.log_prob, q, tf.optimizers.Adam(), 10)

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)>

In [6]:
1

1

## Likelihood

In [7]:
import treeflow.sequences
reload(treeflow.sequences)
import treeflow.substitution_model

subst_model = treeflow.substitution_model.HKY()
category_weights = tf.ones(1)
category_rates = tf.ones(1)

alignment = treeflow.sequences.get_encoded_sequences(fasta_file, taxon_names)
log_prob_conditioned = treeflow.sequences.log_prob_conditioned(alignment, tree['topology'], 1)
                 
def log_likelihood(tree, kappa, frequencies):  
    return log_prob_conditioned(
        subst_model=subst_model,
        category_weights=category_weights,
        category_rates=category_rates,
        branch_lengths=treeflow.sequences.get_branch_lengths(tree, ),
        frequencies=frequencies,
        kappa=kappa
    )
                 
q_sample = q.sample()
                 
log_likelihood(q_sample['tree'], q_sample['kappa'], q_sample['frequencies'])

<tf.Tensor: shape=(), dtype=float32, numpy=nan>